In [ ]:
%matplotlib ipympl
%load_ext autoreload
%autoreload 2

In [ ]:
import cvxpy as cp
import numpy as np
from gcspy import GraphOfConvexPrograms

In [ ]:
graph = GraphOfConvexPrograms()

f1 = graph.add_vertex("f1")
xf1 = f1.add_variable(2)
cf1 = np.array([3, 2.5])
f1.add_constraint(cp.norm_inf(xf1 - cf1) <= .5)

f2 = graph.add_vertex("f2")
xf2 = f2.add_variable(2)
cf2 = np.array([3, -1])
Df2 = np.diag([2, .5])
f2.add_constraint(cp.norm_inf(Df2 @ (xf2 - cf2)) <= 1)

u1 = graph.add_vertex("u1")
xu1 = u1.add_variable(2)
cu1 = np.array([0, .5])
u1.add_constraint(cp.norm2(xu1 - cu1) <= .5)

u2 = graph.add_vertex("u2")
xu2 = u2.add_variable(2)
cu2 = np.array([0, -1])
u2.add_constraint(cp.norm2(xu2 - cu2) <= .5)

u3 = graph.add_vertex("u3")
xu3 = u3.add_variable(2)
cu3 = np.array([0, -2.5])
u3.add_constraint(cp.norm2(xu3 - cu3) <= .5)

In [ ]:
facilities = [f1, f2]
users = [u1, u2, u3]
for facility in facilities:
    for user in users:
        edge = graph.add_edge(facility, user)
        edge.add_cost(cp.norm2(facility.variables[0] - user.variables[0]))

In [ ]:
graph.graphviz()

In [ ]:
prob = graph.solve_facility_location()
print('Problem status:', prob.status)
print('Optimal value:', prob.value)

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.axis("equal")
plt.axis('off')
graph.plot_2d()
graph.plot_2d_subgraph()

# From ILP

In [ ]:
ilp_constraints = []
yv = graph.vertex_binaries()
ye = graph.edge_binaries()

for i, v in enumerate(graph.vertices):
    inc_edges = graph.incoming_indices(v)
    if len(inc_edges) == 0:
        ilp_constraints.append(yv[i] <= 1)
    else:
        ilp_constraints.append(yv[i] == 1)
        ilp_constraints.append(sum(ye[inc_edges]) == 1)
        
for k, edge in enumerate(graph.edges):
    i = graph.vertex_index(edge.tail)
    ilp_constraints.append(yv[i] >= ye[k])

In [ ]:
prob = graph.solve_from_ilp(ilp_constraints)
print('Problem status:', prob.status)
print('Optimal value:', prob.value)